<a href="https://colab.research.google.com/github/getChan/data_campus/blob/master/NLP/SequenceTagger_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. CoNLL-2003 Dataset Download

In [1]:
!wget -O CoNLL-2003.zip https://www.dropbox.com/s/hfr0r95e9ggjozm/CoNLL-2003.zip?dl=0
!mkdir CoNLL-2003
!unzip CoNLL-2003.zip -d CoNLL-2003
!rm CoNLL-2003.zip

--2019-07-31 05:32:45--  https://www.dropbox.com/s/hfr0r95e9ggjozm/CoNLL-2003.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/hfr0r95e9ggjozm/CoNLL-2003.zip [following]
--2019-07-31 05:32:47--  https://www.dropbox.com/s/raw/hfr0r95e9ggjozm/CoNLL-2003.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2f734d0979e524656dbefa5356.dl.dropboxusercontent.com/cd/0/inline/AltLpXRVKKytvdmPhr-afmNMf1WXcwANlzu2RdBokn0HgAOViasvMFIz7MaU1CNHQ4DssJld3Uz3Csw6jMvSMveFbnjDfDccLzCZaxe6FEbN1g/file# [following]
--2019-07-31 05:32:47--  https://uc2f734d0979e524656dbefa5356.dl.dropboxusercontent.com/cd/0/inline/AltLpXRVKKytvdmPhr-afmNMf1WXcwANlzu2RdBokn0HgAOViasvMFIz7MaU1CNHQ4DssJld3Uz3Csw6jMvSMveFbnjDfDccLzCZaxe6FEbN1g/file
Resol

## 2. Import Modules

In [2]:
import json
import collections
from datetime import datetime
import os
import logging
import tensorflow as tf
from nltk.tokenize import word_tokenize
import nltk
import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## 3. Logger Settings

In [0]:
def init_logger(path:str):
    if not os.path.exists(path):
        os.makedirs(path)
    logger = logging.getLogger()
    logger.handlers = []
    logger.setLevel(logging.DEBUG)
    debug_fh = logging.FileHandler(os.path.join(path, "debug.log"))
    debug_fh.setLevel(logging.DEBUG)

    info_fh = logging.FileHandler(os.path.join(path, "info.log"))
    info_fh.setLevel(logging.INFO)

    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)

    info_formatter = logging.Formatter('%(asctime)s | %(levelname)-8s | %(message)s')
    debug_formatter = logging.Formatter('%(asctime)s | %(levelname)-8s | %(message)s | %(lineno)d:%(funcName)s')

    ch.setFormatter(info_formatter)
    info_fh.setFormatter(info_formatter)
    debug_fh.setFormatter(debug_formatter)

    logger.addHandler(ch)
    logger.addHandler(debug_fh)
    logger.addHandler(info_fh)

    return logger


## 4. Hyperparameters Settings

In [0]:
hparams_dict = {
  "root_dir": "out_dirs/KoreaUniv_Data/TEST/",
  "vocab_size": 10000,
  "num_epochs": 10,
  "batch_size": 16,
  "embedding_dim": 100,
  "rnn_hidden_dim": 128,
  "rnn_depth": 3,
  "dropout_keep_prob": 1.0
}

In [8]:
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
root_dir = os.path.join(hparams_dict["root_dir"], "%s/" % timestamp)
logger = init_logger(root_dir)
logger.info("Hyper-parameters: %s" %str(hparams_dict))
hparams_dict["root_dir"] = root_dir
hparams = collections.namedtuple("HParams", sorted(hparams_dict.keys()))(**hparams_dict)

data_dir = "./CoNLL-2003"
dropout_keep_prob_ph = tf.placeholder(tf.float32, shape=[], name="dropout_keep_prob")
logger = logging.getLogger(__name__)
iterator_initializers = []

2019-07-31 05:37:08,286 | INFO     | Hyper-parameters: {'root_dir': 'out_dirs/KoreaUniv_Data/TEST/', 'vocab_size': 10000, 'num_epochs': 10, 'batch_size': 16, 'embedding_dim': 100, 'rnn_hidden_dim': 128, 'rnn_depth': 3, 'dropout_keep_prob': 1.0}


## 5. Make Vocab Table

In [0]:
def make_vocab_table():
    """
    [A]
    Vocabulary(단어집) 파일을 로드합니다.
    단어 -> id, id -> 단어 변환 테이블을 생성합니다.

    """    
    with open(os.path.join(data_dir, "train.vocab"), "r") as _f_handle:
        vocab = [l.strip() for l in list(_f_handle) if len(l.strip()) > 0]
    if len(vocab) > hparams.vocab_size:
        vocab = vocab[:hparams.vocab_size]

    id2word = vocab
    word2id = {}
    for i, word in enumerate(vocab):
        word2id[word] = i


    """
    [B]
    Label(태그 모음) 파일을 로드합니다.
    태그 -> id, id -> 태그 변환 테이블을 생성합니다.

    """

    with open(os.path.join(data_dir, "label.vocab"), "r") as _f_handle:
      labels = [l.strip() for l in list(_f_handle) if len(l.strip()) > 0]
      labels.insert(0, "PAD")
      id2label = labels
      label2id = {}
      for i, label in enumerate(labels):
          label2id[label] = i

    return (id2word, word2id), (id2label, label2id)


## 6. Build Graph (Sequence Tagger Model)

In [0]:
def build_graph(inputs:tf.Tensor, lengths:tf.Tensor, id2word, id2label):
      print("Building graph for model: sequence tagger")

      """
      [C]
      단어 임베딩 행렬을 생성합니다.
      단어 id를 단어 임베딩 텐서로 변환합니다.
      """
 
      # Number of possible output categories.
      output_dim = len(id2label)
      vocab_size = len(id2word) + 1
      embeddings = tf.get_variable(
          "embeddings",
          shape=[vocab_size, hparams.embedding_dim],
          initializer=tf.initializers.variance_scaling(
              scale=1.0, mode="fan_out", distribution="uniform")
      )
      embedded = tf.nn.embedding_lookup(embeddings, inputs)
      # shape = [batch_size, sequence_length: batch마다 이것보다 작은 문장들은 padding, embed_dim:100] : inputs
      layer_out = embedded


      """
      [D]
      단어 임베딩을 RNN의 입력으로 사용하기 전,
      차원 수를 맞춰주고 성능을 향상시키기 위해
      projection layer를 생성하여 텐서를 통과시킵니다.
      """

      # batch, seqence_length, embedding_dim -> batch, sequence_length, rnn_hidden_dim
      layer_out = tf.layers.dense(
          inputs=layer_out,
          units=hparams.rnn_hidden_dim,
          activation=tf.nn.relu,
          kernel_initializer=tf.initializers.variance_scaling(
              scale=1.0, mode="fan_avg", distribution="normal"),
          name="input_projection"
      )


      """
      [E]
      양방향 RNN을 생성하고, 여기에 텐서를 통과시킵니다.
      이렇게 하여, 단어간 의존 관계가 반영된 단어 자질 텐서를 얻습니다.
      """
      with tf.variable_scope("bi-RNN"):
          # Build RNN layers
          rnn_cell_forward = tf.contrib.rnn.LSTMCell(hparams.rnn_hidden_dim)
          rnn_cell_backward = tf.contrib.rnn.LSTMCell(hparams.rnn_hidden_dim)

          # Apply dropout to RNN
          if hparams.dropout_keep_prob < 1.0:
              rnn_cell_forward = tf.contrib.rnn.DropoutWrapper(rnn_cell_forward, output_keep_prob=dropout_keep_prob_ph)
              rnn_cell_backward = tf.contrib.rnn.DropoutWrapper(rnn_cell_backward, output_keep_prob=dropout_keep_prob_ph)

          # Stack multiple layers of RNN
          # rnn_cell_forward = tf.contrib.rnn.MultiRNNCell([rnn_cell_forward] * hparams.rnn_depth)
          # rnn_cell_backward = tf.contrib.rnn.MultiRNNCell([rnn_cell_backward] * hparams.rnn_depth)

          (output_forward, output_backward), _ = tf.nn.bidirectional_dynamic_rnn(
              rnn_cell_forward, rnn_cell_backward,
              inputs=layer_out,
              sequence_length=lengths, # embedding lookup에서 다 padding 해서 length 같지 않나?
              dtype=tf.float32
          )
          
          
          hiddens = tf.concat([output_forward, output_backward], axis=-1)
          # shape = [batch_size, max_sequence_length, rnn_hidden_dim*2] # backward vector concat하므로  *2 된 vector


      """
      [F]
      마스킹을 적용하여 문장 길이를 통일하기 위해 적용했던 padding을 제거합니다.
      """
      
      # Donald Trump is the president of the United States
      # Barack Obama was the president of the United States <PAD> <PAD>....
      # I live in paris <PAD> <PAD> ,.....
      
      
      # [3, 10, 100](embedding) -> [3,10,128] (input_projection) -> [3,10,128](rnn_forward) , [3,10,128](rnn_forward) ->  [3,10,256] 
      # -> [3,10,10]
            
      # [10, 6, 5] sequence_mask 통과시키면
      # -> [[TRUE] * 10,
      #     [TRUE] * 6 + [FALSE] * 4...] 이렇게 변한다.
      
      
      mask = tf.sequence_mask(lengths)
      
      ## boolean mask : TRUE에 해당되는 것만 가져와라
      bi_lstm_out = tf.reshape(tf.boolean_mask(hiddens, mask), [-1, hparams.rnn_hidden_dim * 2])
      layer_out = bi_lstm_out  # shape=[sum of seq length, 2*LSTM hidden layer size]
      # [21,256] W: [256,10], b: [10] ->[21,10] 왜 21? 패딩을 없어주면 10, 5, 6 나오기 때문임.

      """
      [G]
      단어 자질 텐서를 바탕으로 단어의 태그를 예측합니다.
      이를 위해 fully-connected(dense) layer를 생성하고 텐서를 통과시킵니다.
      """

      with tf.variable_scope("read-out"):
        prev_layer_size = layer_out.get_shape().as_list()[1]
        weight = tf.get_variable("weight", shape=[prev_layer_size, output_dim],
                                 initializer=tf.initializers.variance_scaling(
                                     scale=2.0, mode="fan_in", distribution="normal"
                                 ))
        bias = tf.get_variable("bias", shape=[output_dim],
                               initializer=tf.initializers.zeros())
        predictions = tf.add(tf.matmul(layer_out, weight), bias, name='predictions')


      return predictions


## 7. Load Data (tf.data)

In [0]:
def load_data(id2word, word2id, id2label, label2id):
      """
      [L]
      단어->id 및 태그->id 변환 테이블을 텐서 그래프에 추가합니다.
      """
      word2id = tf.contrib.lookup.index_table_from_tensor(
        mapping=tf.constant(id2word),
        num_oov_buckets=1,
        name="word2id"
      )

      label2id = tf.contrib.lookup.index_table_from_tensor(
        mapping=tf.constant(id2label),
        default_value=label2id["O"],
        name="label2id"
      )

      """
      [M]
      입력 데이터 파일을 읽어들여 이를 단어 id로 변환하는 텐서 그래프를 생성합니다.
      """
      input_dataset = tf.data.TextLineDataset(os.path.join(data_dir, "train.inputs"))
      batched_input_dataset = input_dataset.batch(hparams.batch_size)
      input_iterator = batched_input_dataset.make_initializable_iterator()
      batch_input = input_iterator.get_next()
      batch_input.set_shape([hparams.batch_size])
      words = tf.string_split(batch_input, " ")
      word_ids = word2id.lookup(words)
      dense_word_ids = tf.sparse_tensor_to_dense(word_ids)
      # shape = [batch_size, time]


      line_number = word_ids.indices[:, 0]
      line_position = word_ids.indices[:, 1]
      lengths = tf.segment_max(data=line_position,
                               segment_ids=line_number) + 1


      """
      [N]
      태그 데이터 파일을 읽어들여 이를 태그 id로 변환하는 텐서 그래프를 생성합니다.
      """

      label_dataset = tf.data.TextLineDataset(os.path.join(data_dir, "train.labels"))
      batched_label_dataset = label_dataset.batch(hparams.batch_size)
      label_iterator = batched_label_dataset.make_initializable_iterator()
      batch_label_str = label_iterator.get_next()
      batch_label = tf.string_split(batch_label_str, " ")
      label_ids = label2id.lookup(batch_label)
      dense_label_ids = tf.sparse_tensor_to_dense(label_ids)
      # shape = [batch_size, time]

      mask = tf.sequence_mask(lengths)
      dense_label_ids = tf.boolean_mask(dense_label_ids, mask)

      iterator_initializers.append(input_iterator.initializer)
      iterator_initializers.append(label_iterator.initializer)




      return dense_word_ids, dense_label_ids, lengths

## 8. Train Model (session call)

In [0]:
  def train_model():
        sess = tf.Session()
        with sess.as_default():
            global_step = tf.Variable(0, name='global_step', trainable=False)
            # vocab 만드는 부분
            (id2word, word2id), (id2label, label2id) = make_vocab_table()
            
            # Data loading
            inputs, labels, lengths = load_data(id2word, word2id, id2label, label2id)

            # 실제 딥러닝 모델 구현
            with tf.variable_scope("build_graph", reuse=False):
                logits = build_graph(inputs, lengths, id2word, id2label)

            """
            [O]
            모델을 훈련시키기 위해 필요한 오퍼레이션들을 텐서 그래프에 추가합니다.
            여기에는 loss, train, accuracy 계산 등이 포함됩니다.
            """
            
            loss_op = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels,
                                                                     name="cross_entropy")
            loss_op = tf.reduce_mean(loss_op, name='cross_entropy_mean')
            train_op = tf.train.AdamOptimizer().minimize(loss_op, global_step=global_step)

            eval = tf.nn.in_top_k(logits, labels, 1)
            correct_count = tf.reduce_sum(tf.cast(eval, tf.int32))
            accuracy = tf.divide(correct_count, tf.shape(labels)[0])


            # Initialize iterators, tables, and variables.
            local_iterator_initializers = tf.group(*iterator_initializers)
            tf.tables_initializer().run()
            tf.global_variables_initializer().run()

            saver = tf.train.Saver()

            for epochs_completed in range(hparams.num_epochs):
                local_iterator_initializers.run()
                accuracy_mean, loss_mean, idx_cnt = 0, 0, 0
                while True:
                    """
                    [P]
                    그래프에 데이터를 입력하여 필요한 계산들을 수행하고,
                    Loss에 따라 gradient를 계산하여 파라미터들을 업데이트합니다.
                    이러한 과정을 training step이라고 합니다.
                    """
                    try:
                      accuracy_val, label_ids_val, loss_val, global_step_val, _ = sess.run(
                          [accuracy, labels, loss_op, global_step, train_op],
                          feed_dict={dropout_keep_prob_ph: hparams.dropout_keep_prob}
                      )
                      accuracy_mean += accuracy_val
                      loss_mean += loss_val
                      idx_cnt += 1
                      if global_step_val % 50 == 0:
                          accuracy_mean /= idx_cnt
                          loss_mean /= idx_cnt
                          logger.info("[Step %d] loss: %.4f, accuracy: %.2f%%" % (global_step_val, loss_mean, accuracy_mean * 100))
                          accuracy_mean, loss_mean,idx_cnt = 0, 0, 0
                    except tf.errors.OutOfRangeError:
                      # End of epoch.
                      break

                """
                [Q]
                전체 학습 데이터에 대하여 1회 학습을 완료하였습니다.
                이를 1 epoch라고 합니다.
                딥러닝 모델의 학습은 일반적으로 수십~수백 epoch 동안 진행됩니다.
                
                """
                logger.info("End of epoch %d." % (epochs_completed+1))
                save_path = saver.save(sess, "saves/model.ckpt", global_step=global_step_val)
                logger.info("Model saved at: %s" % save_path)

## Train the model

In [17]:
# Train the vanilla Bi-directional LSTM model
train_model()

2019-07-31 06:10:25,238 | WARNING  | 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

2019-07-31 06:10:25,469 | WARNING  | From <ipython-input-15-6b9277005a04>:24: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
2019-07-31 06:10:25,490 | WARNING  | From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/lookup_ops.py:978: add

Building graph for model: sequence tagger


2019-07-31 06:10:25,955 | WARNING  | From <ipython-input-14-c485357300ab>:48: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
2019-07-31 06:10:25,957 | WARNING  | From <ipython-input-14-c485357300ab>:64: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
2019-07-31 06:10:25,962 | WARNING  | From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/rnn.py:464: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
2019-07-31 06:10:26,066 | WARNING  | From /usr/loc

## Model Load and Predict

In [0]:
def load_and_predict(saved_file:str):
    sentence = input("Enter a sentence: ")

    """
    [H]
    입력 문자열을 단어/문장부호 단위로 쪼개고, 이를 다시 단어 id로 변환합니다.
    """
    sentence = word_tokenize(sentence)
    word_ids = []
    (id2word, word2id), (id2label, label2id) = make_vocab_table()

    for word in sentence:
        if word in word2id:
            word_ids.append(word2id[word])
        else:
            word_ids.append(len(word2id))

    
    tf.reset_default_graph()
    sess = tf.Session()
    with sess.as_default():
        """
        [I]
        태깅을 수행하기 위해 텐서 그래프를 생성합니다.
        """
        dense_word_ids = tf.constant(word_ids)
        lengths = tf.constant(len(word_ids))
        # Insert batch dimension.
        dense_word_ids = tf.expand_dims(dense_word_ids, axis=0)
        lengths = tf.expand_dims(lengths, axis=0)

        with tf.variable_scope("build_graph", reuse=tf.AUTO_REUSE):
            logits = build_graph(dense_word_ids, lengths, id2word, id2label)
        predictions = tf.argmax(logits, axis=1)

        """
        [J]
        저장된 모델을 로드하고, 데이터를 입력하여 태깅 결과를 얻습니다.
        """
        saver = tf.train.Saver()
        saver.restore(sess, saved_file)
        pred_val = sess.run(
            [predictions]
        )[0]


    """
    [K]
    태깅 결과를 출력합니다.
    """
    pred_str = [id2label[i] for i in pred_val]
    for word, tag in zip(sentence, pred_str):
        print("%s[%s]" %(word, tag), end=' ')


In [20]:
load_and_predict("/content/saves/model.ckpt-5622")

Enter a sentence: where is my book?
Building graph for model: sequence tagger


2019-07-31 06:32:18,343 | INFO     | Restoring parameters from /content/saves/model.ckpt-5622


where[O] is[O] my[O] book[O] ?[O] 